In [1]:
import random
import os
import glob
import shutil

In [2]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'MAC_Macao'

In [3]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/MAC_Macao/figures/MAC_CO2_totals.png',
 '../country_data/MAC_Macao/figures/MAC_Minx_top20_subsectors.png',
 '../country_data/MAC_Macao/figures/MAC_EIA_1.png',
 '../country_data/MAC_Macao/figures/MAC_CDIAC_1.png',
 '../country_data/MAC_Macao/figures/MAC_GCP_1.png']

In [4]:
random_fig = random.choice(figures)
random_fig

'../country_data/MAC_Macao/figures/MAC_CO2_totals.png'

In [5]:
shutil.copyfile(random_fig, "../supplemental/random_daily_fig.png",)

'../supplemental/random_daily_fig.png'

In [6]:
print("random_fig path imported")

random_fig path imported
